In [1]:
#imports
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
import chromadb

/home/chahar/miniconda3/envs/food/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Connecting to ChromaDB collection

In [2]:
# Reuse the same model used for embedding food data
embedding_model = HuggingFaceEmbeddings(model_name="clip-ViT-B-32")

# Connect to local ChromaDB where you stored food vectors
vectorstore = Chroma(
    client=chromadb.PersistentClient(path="./chroma_db"),
    collection_name="food_macros_clip",
    embedding_function=embedding_model
)

/tmp/ipykernel_4810/2157315273.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="clip-ViT-B-32")
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/tmp/ipykernel_4810/2157315273.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists 

Buffer Memory setup

In [3]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

/tmp/ipykernel_4810/2971518456.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Retrieval LLM agent

In [4]:
llm = Ollama(model="llama2")

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
    memory=memory,
    return_source_documents=True,
    output_key="answer"
)

/tmp/ipykernel_4810/103396959.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2")


In [5]:
print("🤖 Chat with your Food Macros Agent (type 'exit' to quit):\n")
while True:
    question = input("You: ")
    if question.lower() in ['exit', 'quit']:
        break

    result = qa_chain({"question": question})
    print(f"AI: {result['answer']}\n")

🤖 Chat with your Food Macros Agent (type 'exit' to quit):



/tmp/ipykernel_4810/271645556.py:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"question": question})


AI: Based on the context provided, here are some foods that are high in protein and low in fat:

1. Chicken breast: Chicken breast is a lean protein source that contains approximately 30-40 grams of protein per 3-ounce serving, with almost no fat.
2. Turkey breast: Similar to chicken breast, turkey breast is a lean protein source with about 30-40 grams of protein per 3-ounce serving and very little fat.
3. Fish (such as salmon, tuna, and cod): Many types of fish are low in fat and high in protein, with approximately 20-30 grams of protein per 3-ounce serving.
4. Tofu: Tofu is made from soybeans and contains approximately 20 grams of protein per 3-ounce serving, with very little fat.
5. Legumes (such as lentils, chickpeas, and black beans): Legumes are high in protein and low in fat, with approximately 15-20 grams of protein per 3-ounce serving.
6. Egg whites: Egg whites are almost pure protein and contain very little fat, with approximately 3-4 grams of protein per large egg.
7. Greek 

CHAT HISTORY

In [6]:
for msg in memory.chat_memory.messages:
    print(f"{msg.type.upper()}: {msg.content}")

HUMAN: which food have high protein and low fat?
AI: Based on the context provided, here are some foods that are high in protein and low in fat:

1. Chicken breast: Chicken breast is a lean protein source that contains approximately 30-40 grams of protein per 3-ounce serving, with almost no fat.
2. Turkey breast: Similar to chicken breast, turkey breast is a lean protein source with about 30-40 grams of protein per 3-ounce serving and very little fat.
3. Fish (such as salmon, tuna, and cod): Many types of fish are low in fat and high in protein, with approximately 20-30 grams of protein per 3-ounce serving.
4. Tofu: Tofu is made from soybeans and contains approximately 20 grams of protein per 3-ounce serving, with very little fat.
5. Legumes (such as lentils, chickpeas, and black beans): Legumes are high in protein and low in fat, with approximately 15-20 grams of protein per 3-ounce serving.
6. Egg whites: Egg whites are almost pure protein and contain very little fat, with approximat